In [42]:
from bs4 import BeautifulSoup
import requests
import re

# Url of website
url='https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411'
# I send my HTTP request with a "GET" to the site server to identify in the url
r = requests.get(url)
# I display the requested url and the return of the server
print(url, r.status_code)
# I ask beautifulSoup to keep in a soup variable the web page to scrape (url) an html script
soup = BeautifulSoup(r.content,'lxml')
soup

property_dict = {}

for elem in soup.find_all('tr'):
    if elem.th:
        if elem.th.string:
            if elem.th.string.strip() == "Price":
                price_descendants = [descendants for descendants in elem.td.descendants]
                r = re.compile("(€\s(\d+)+,+\d{3})")
                [filtered_descendant for filtered_descendant in filter(r.search, price_descendants)]
                print(r.search([filtered_descendant for filtered_descendant in filter(r.search, price_descendants)][0])[0])
            else:
                property_dict[elem.th.string.strip()] = elem.td.contents[0].strip()

https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411 200


TypeError: expected string or bytes-like object

In [13]:
from bs4 import BeautifulSoup
import re
import requests

def scrap(url):
    dict = {}
    r = requests.get(url)
    soup = BeautifulSoup(r.content,'html.parser')

    #dict["locality"] = 'k'
    dict["postcode"] = re.findall('/[0-9]{4}/', url)[0].replace('/', '')
        
    return dict

my_url = "https://www.immoweb.be/en/classified/house/for-sale/seraing/4100/8771411"
scrap(my_url)



{'postcode': '4100'}

In [34]:
lst = ["blabla\n € 123,000\n","whatever"]
r = re.compile("(€\s(\d+)+,+\d{3})")
r.search([matched_element for matched_element in filter(r.search, lst)][0])[0]

'€ 123,000'